# Quantum Teleporation Application

In this demo, we'll demonstrate the simulation of the teleportation application. The network topology will have two nodes: Alice and Bob. Alice will teleport a qubit to Bob. 

## 1. The Basics

The figure below shows the quantum circuit of teleportation, as well as showing which qubits belong to Alice and which qubit belongs to Bob.

<img src="figures/teleport-circuit.png" width="600">

**Set up**: Assume node Alice and Bob both have two quantum memories, one labeled as data memory and the other labeled as communication memory. There is a quantum channel between Alice and Bob, and the BSM node is in the middle of the quantum channel.

Let's break down quantum teleportation into 4 steps.

**Step 1**: The tradition is to let Alice send a qubit to Bob. So, Alice prepares $\ket{\psi}$ at memory $d_1$.

<img src="figures/teleport-1.png" width="600">

**Step 2**: Generate an entangled pair between Alice's memory $c_1$ and Bob's memory $c_2$.

<img src="figures/teleport-2.png" width="600">

**Step 3**: Alice does a Bell measurement on it's memory $d_1$ and $c_1$. The measurement results are sent to Bob.

<img src="figures/teleport-3.png" width="600">

**Step 4**: Bob does the correction on $c_2$ upon receiving the measurement results. Then $\ket{\psi}$ is teleported to Bob's memory $c_2$.

<img src="figures/teleport-4.png" width="600">

## 2. Show Me The Code

**The imports**

- `log`: defines the behavior for the default SeQUeNCe logging system
- `RequestApp`: is an example application included with SeQUeNCe. We will investigate its behavior through a class that inherits from this
- `DQCNode`: Node that supports Distributed Quantum Computing
- `TeleportProtocol`: A network protocol that does the teleportation
- `TeleportMessage`: Classical message used to convey the Pauli corrections (x, z) from sender to receiver during teleportation.
- `DQCNetTopo`: Class for generating a distributed quantum computing network with distributed quantum computing nodes
- `random_state`: Generate a random quantum state

In [1]:
import numpy as np
import sequence.utils.log as log
from sequence.app.request_app import RequestApp
from sequence.topology.node import DQCNode
from sequence.entanglement_management.teleportation import TeleportProtocol, TeleportMessage
from sequence.topology.dqc_net_topo import DQCNetTopo
from sequence.utils.random_state import random_state
from sequence.constants import MILLISECOND

### 2.1 The Teleportation App

The class `TeleportApp` is responsible for managing quantum teleportation between quantum nodes. It utilizes the `TeleportProtocol` to handle the teleportation process, including the reservation of entangled pairs and the application of corrections based on classical messages.

In [2]:
class TeleportApp(RequestApp):
    """Code for the teleport application.

    TeleportApp is a specialized RequestApp that implements quantum teleportation.
    It handles the teleportation protocol between two quantum nodes (Alice and Bob).

    Attributes:
        node (DQCNode): The quantum node this app is attached to.
        name (str): The name of the teleport application.
        results (list): List to store the results of teleportation.
        teleport_protocol (TeleportProtocol): The teleportation protocol instance.
    """
    def __init__(self, node: DQCNode):
        super().__init__(node)
        self.name = f"{self.node.name}.TeleportApp"
        node.teleport_app = self   # register ourselves so incoming TeleportMessage lands here:
        self.results = []          # where we’ll collect Bob’s teleported state
        self.teleport_protocol = TeleportProtocol(owner=node, data_src=None)  # create a single protocol instance, on both Alice & Bob

    def start(self, responder: str, start_t: int, end_t: int, memory_size: int, fidelity: float, data_src: int):
        """Start the teleportation process.

        Args: 
            responder (str): Name of the responder node (Bob).
            start_t (int): Start time of the teleportation (in ps).
            end_t (int): End time of the teleportation (in ps).
            memory_size (int): Size of the memory used for the teleportation.
            fidelity (float): Target fidelity of the teleportation.
            data_src (int): Index of the data qubit to be teleported.
        """
        # configure our single protocol
        self.teleport_protocol.data_src     = data_src
        self.teleport_protocol.is_initiator = (self.node.name != responder)
        self.teleport_protocol.remote       = responder

        # cache Alice’s data‐qubit key for the Bell measurement
        memory_array = self.node.get_component_by_name(self.node.data_memo_arr_name)
        data_memory = memory_array[data_src]
        self.teleport_protocol.data_memory = data_memory
        self.teleport_protocol._q_data     = data_memory.qstate_key

        log.logger.debug(f"{self.name}: start() → responder={responder}, data_src={data_src}")

        # reserve and generate EPR pair
        super().start(responder, start_t, end_t, memory_size, fidelity)

        # notify the protocol we’re kicking off
        self.teleport_protocol.start()

    def get_reservation_result(self, reservation, result: bool):
        """Handle the reservation result from the network manager.

        Args:
            reservation (Reservation): The reservation object.
            result (bool): True if the reservation was successful, False otherwise.
        """
        super().get_reservation_result(reservation, result)
        log.logger.debug(f"{self.name}: reservation_result → {result}")

    def get_memory(self, info):
        """Handle memory state changes.

        Args:
            info (MemoryInfo): Information about the memory state change.
        """
        # once we see our entangled half, hand it to the protocol
        if info.index in self.memo_to_reservation:
            if info.state == "ENTANGLED":
                self.teleport_protocol.handle_entangled(info, self.memo_to_reservation[info.index])

    def received_message(self, src: str, msg: TeleportMessage):
        """Handle incoming teleport messages.

        Args:
            src (str): Source node name.
            msg (TeleportMessage): The teleport message received.
        """
        # feed Bob’s corrections into the protocol
        self.teleport_protocol.received_message(src, msg)

    def teleport_complete(self, comm_key: int):
        """Called by TeleportProtocol once Bob's qubit is corrected. comm_key holds the teleported |ψ⟩.

        Args:
            comm_key (int): The key of the comm memory where the teleported state is stored.
        """
        my_qubit  = self.node.timeline.quantum_manager.get(comm_key)
        psi = my_qubit.state # get qubit state
        self.results.append(psi) #append result to the list


### 2.2 The teleport() function

Teleport a $\ket{\psi}$ from Alice to Bob

In [3]:
def teleport(psi: np.array) -> np.array:
    """Do a teleportation
    Args:
        psi (np.array): the state to be teleported at Alice
    Return:
        np.array: the state received at Bob
    """
    
    topo = DQCNetTopo("teleport_2node.json")
    tl   = topo.tl

    log_filename = 'teleport.log'
    log.set_logger(__name__, tl, log_filename)
    log.set_logger_level('INFO')
    modules = ['generation', 'teleportation']
    for module in modules:
        log.track_module(module)

    nodes = topo.nodes[DQCNetTopo.DQC_NODE]
    alice = next(n for n in nodes if n.name=="router_0")
    bob   = next(n for n in nodes if n.name=="router_1")
    
    # 1) Prepare |ψ> in Alice’s data qubit
    data_memo_arr = alice.get_component_by_name(alice.data_memo_arr_name)
    data_memo_arr[0].update_state(psi)

    # 2) Attach the TeleportApp on both nodes
    alice_app = TeleportApp(alice)
    bob_app   = TeleportApp(bob)

    # 3) Kick off teleport
    alice_app.start(
        responder   = bob.name,
        start_t     = 10  * MILLISECOND,
        end_t       = 30 * MILLISECOND,
        memory_size = 1,
        fidelity    = 0.8,
        data_src    = 0
    )
    
    # 4) Run the simulation
    tl.init()
    tl.run()
    
    # 5) Read out Bob’s data qubit state
    teleported_qubit = bob_app.results[0]

    return np.array(teleported_qubit)

### 2.3 The main() function

In [4]:
def main():
    psi = np.array(random_state())
    
    teleported_psi = teleport(psi)  # run the teleporation
    
    print(f'psi            = {psi}')
    print(f'teleported psi = {teleported_psi}')
    
    assert teleported_psi.shape == psi.shape
    assert np.allclose(teleported_psi, psi, atol=1e-6), f"teleported state {teleported_psi} != original {psi}"
    
    print('\nTeleporation success!')

In [5]:
# please run this cell repeatedly
main()

psi            = [ 0.70368352+0.j         -0.25830117+0.66189879j]
teleported psi = [ 0.70368352+0.j         -0.25830117+0.66189879j]

Teleporation success!


**NOTE**: Please checkout the `teleport.log` file go see the events being logged.